In [1]:
import mongo
from pymatgen.io.ase import AseAtomsAdaptor
import pickle

CO_docs = pickle.load(open('/home/zulissi/CO_docs.pkl','rb'))
CO_docs = CO_docs[:200]

#Could be implemented as a transform if needed
structure_list = [AseAtomsAdaptor.get_structure(mongo.make_atoms_from_doc(doc)) for doc in CO_docs]
target_list = [doc['energy'] for doc in CO_docs]


In [2]:
from cgcnn.data import StructureData
import torch
import numpy as np

#X
dataset = StructureData(structure_list, target_list, '/home/zulissi/software/cgcnn_sklearn/atom_init.json')
#y, energies
y = torch.Tensor(np.array(target_list).reshape((len(target_list),1)))


In [3]:
from cgcnn.model import CrystalGraphConvNet
from sklearn.datasets import make_classification
from torch import nn
import torch.nn.functional as F

from skorch import NeuralNetRegressor

structures, _ = dataset[0]
orig_atom_fea_len = structures[0].shape[-1]
nbr_fea_len = structures[1].shape[-1]

from torch.utils.data import Dataset, DataLoader
from cgcnn.data import collate_pool

net = NeuralNetRegressor(
    CrystalGraphConvNet,
    iterator_train = DataLoader,
    iterator_valid = DataLoader,
    module__orig_atom_fea_len = orig_atom_fea_len,
    module__nbr_fea_len = nbr_fea_len,
    iterator_train__num_workers=4,
    iterator_train__pin_memory=True,
    iterator_train__collate_fn = collate_pool,
    iterator_valid__pin_memory=True,
    iterator_valid__collate_fn = collate_pool,
    max_epochs=10,
    lr=0.1,
    device='cuda'
)

In [4]:
net.fit(dataset,y=y)

  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1      907.7483     2404.1331  39.8048
      2       75.2742       34.2038  29.4979
      3       26.0986       14.8460  29.6444
      4       11.0126        6.7142  29.6744
      5        4.8284        3.2526  29.7986
      6        2.2908        1.7503  29.8559
      7        1.2483        1.0807  29.6812
      8        0.8192        0.7716  29.6743
      9        0.6420        0.6227  29.8668
     10        0.5686        0.5475  29.6978


<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=CrystalGraphConvNet(
    (embedding): Linear(in_features=92, out_features=64, bias=True)
    (convs): ModuleList(
      (0): ConvLayer(
        (fc_full): Linear(in_features=169, out_features=128, bias=True)
        (sigmoid): Sigmoid()
        (softplus1): Softplus(beta=1, threshold=20)
        (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (softplus2): Softplus(beta=1, threshold=20)
      )
      (1): ConvLayer(
        (fc_full): Linear(in_features=169, out_features=128, bias=True)
        (sigmoid): Sigmoid()
        (softplus1): Softplus(beta=1, threshold=20)
        (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (softplus2): Sof